In [1]:
import pandas as pd
books = pd.read_csv("../datasets/books_with_emotion.csv")
books

,BookID,FormattedBookID,Book,Author,Description_x,Genres,Avg_Rating,Num_Ratings,URL,Filtered_Genres,Description_y,Emotion,sentiment
0,1,BK000001,Harry Potter and the Philosopher’s Stone (Harr...,J.K. Rowling,Harry Potter thinks he is an ordinary boy - un...,"['Fantasy', 'Fiction', 'Young Adult', 'Magic',...",4.47,9278135,https://www.goodreads.com/book/show/72193.Harr...,"['Fantasy', 'Fiction', 'Young Adult', 'Magic',...",Harry Potter thinks he is an ordinary boy - un...,"['fear', 0.774424135684967]",fear
1,2,BK000002,"Hungerspelen (The Hunger Games, #1)",Suzanne Collins,24 deltagare. Bara en överlever. Varje år dire...,"['Young Adult', 'Fiction', 'Dystopia', 'Fantas...",4.33,7963186,https://www.goodreads.com/book/show/13616052-h...,"['Young Adult', 'Fiction', 'Dystopia', 'Fantas...",24 deltagare. Bara en överlever. Varje år dire...,"['fear', 0.8872978687286377]",fear
2,3,BK000003,"The Hunger Games (The Hunger Games, #1)",Suzanne Collins,"Could you survive on your own in the wild, wit...","['Young Adult', 'Fiction', 'Dystopia', 'Fantas...",4.33,7963186,https://www.goodreads.com/book/show/12187803-t...,"['Young Adult', 'Fiction', 'Dystopia', 'Fantas...","Could you survive on your own in the wild, wit...","['neutral', 0.943543553352356]",neutral
3,4,BK000004,"The Hunger Games (The Hunger Games, #1)",Suzanne Collins,In the ruins of a place once known as North Am...,"['Young Adult', 'Fiction', 'Dystopia', 'Fantas...",4.33,7963186,https://www.goodreads.com/book/show/18626858-t...,"['Young Adult', 'Fiction', 'Dystopia', 'Fantas...",In the ruins of a place once known as North Am...,"['sadness', 0.7970337271690369]",sadness
4,5,BK000005,"The Hunger Games (The Hunger Games, #1)",Suzanne Collins,Sometime in the future after the destruction o...,"['Young Adult', 'Fiction', 'Dystopia', 'Fantas...",4.33,7963186,https://www.goodreads.com/book/show/12091570-t...,"['Young Adult', 'Fiction', 'Dystopia', 'Fantas...",Sometime in the future after the destruction o...,"['neutral', 0.8030902743339539]",neutral
...,...,...,...,...,...,...,...,...,...,...,...,...,...
8840,9026,BK009026,"Mutantupproret 99,4% DNA",Martin J. Sallberg,En dystopisk framtidsskildring av en värld eft...,"['Science Fiction', 'Dystopia']",4.20,5,https://www.goodreads.com/book/show/29762076-m...,"['Science Fiction', 'Dystopia']",En dystopisk framtidsskildring av en värld eft...,"['neutral', 0.5866175293922424]",neutral
8841,9027,BK009027,The Future of Man: Extinction or Glory?,Peter Hollings,"In only a few generations, our species will ei...",['Science'],4.40,5,https://www.goodreads.com/book/show/6586918-th...,['Science'],"In only a few generations, our species will ei...","['neutral', 0.966056227684021]",neutral
8842,9030,BK009030,"The Stone Seekers (Dragon Hunters, #1)",L.L. Hansen,"In 1000 B.C., two 10-year-old boys, Zeth and C...","['Fantasy', 'Dragons']",4.00,4,https://www.goodreads.com/book/show/17259389-t...,['Fantasy'],"In 1000 B.C., two 10-year-old boys, Zeth and C...","['fear', 0.8579759001731873]",fear
8843,9033,BK009033,The Travels of Marco Polo,Giulio Einaudi,"Marco Polo, a Medieval Venetian, traveled thro...","['Travel', 'History']",3.00,4,https://www.goodreads.com/book/show/15790154-t...,['History'],"Marco Polo, a Medieval Venetian, traveled thro...","['neutral', 0.9289608001708984]",neutral


In [36]:
df = books[["FormattedBookID","URL"]]

In [28]:
import numpy as np
from bs4 import BeautifulSoup
import requests

headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3'
}

In [46]:
import requests
from bs4 import BeautifulSoup
from concurrent.futures import ThreadPoolExecutor, as_completed
import time
from tqdm import tqdm
import pandas as pd

# Custom headers to avoid being blocked
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3'
}

# Function to scrape the book image URL
def scrape_images(url):
    try:
        response = requests.get(url, headers=headers, timeout=10)
        if response.status_code == 200:
            soup = BeautifulSoup(response.text, 'html.parser')
            og_image_tag = soup.find('meta', {'property': 'og:image'})
            if og_image_tag:
                return og_image_tag['content']
        return "Not Found"
    except requests.exceptions.RequestException:
        return "Request Failed"

In [ ]:
url = "https://www.goodreads.com/book/show/72193.Harry_Potter_and_the_Philosopher_s_Stone"
scrape_images(url)

In [ ]:
# Number of threads for parallel processing
MAX_THREADS = 20  
# Progress bar setup
tqdm.pandas()

# Start timing
start_time = time.time()

# Run scraping in parallel
image_results = []
with ThreadPoolExecutor(max_workers=MAX_THREADS) as executor:
    future_to_url = {executor.submit(scrape_images, url): url for url in df["URL"]}
    
    for future in tqdm(as_completed(future_to_url), total=len(df), desc="Scraping Images"):
        url = future_to_url[future]
        try:
            image_url = future.result()
        except Exception:
            image_url = "Error"
        image_results.append((url, image_url))

# Convert results to a dictionary and update the DataFrame
image_dict = dict(image_results)
df["Img_Path"] = df["URL"].map(image_dict)

# End timing
end_time = time.time()
total_time = end_time - start_time

print(f"\n✅ Scraped {len(df)} books in {total_time:.2f} seconds using {MAX_THREADS} threads!")
print(df.head())  # Show the first few rows

In [49]:
df.to_csv("Books_Img_Path",index=False)

In [ ]:
df["Img_Path"][0]

In [ ]:
df

In [ ]:
books = pd.merge(df[["FormattedBookID","Img_Path"]],books,on="FormattedBookID")

In [ ]:
books.to_csv("books_with_image_paths.csv",index=False)